## Freemium Estadística para Data Analysis
#### Explorando datos
---
El hundimiento del Titanic es uno de los naufragios más infames de la historia.

El 15 de abril de 1912, durante su viaje inaugural, el ampliamente considerado "insumergible" RMS Titanic se hundió después de chocar con un iceberg. Desafortunadamente, no había suficientes botes salvavidas para todos a bordo, lo que resultó en la muerte de 1502 de 2224 pasajeros y tripulantes.

Si bien hubo algún elemento de suerte involucrado en la supervivencia, parece que algunos grupos de personas tenían más probabilidades de sobrevivir que otros.

Durante este curso, utilizaremos datos reales del Titanic para **calcular probabilidades y expectativas condicionales, de los pasajeros.**

Como comentamos en los videos, el primer paso de cualquier analista de datos es explorar, entender y limpiar los datos. **Todo analysis comienza con datos.**

Para este análisis, no es necesario saber programar, ni usar python. Pero por simplicidad de visualización haremos este ejercicio usando Python 3.7 y la librería Pandas.

### Explorar datos
---
Nuestro primer paso es entender qué tipos de datos y estructura estaremos trabajando, por lo tanto lo primero que haremos es leer el archivo csv. Puedes encontrar los datos, junto a una descripción en Kaggle, [data](https://www.kaggle.com/c/titanic/data)

In [1]:
import pandas as pd
import locale

In [2]:
csv_file = 'titanic.csv'
df = pd.read_csv(csv_file, encoding='iso-8859-1')

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


Podemos ver que estamos lidiando con valores enteros, objetos y decimales. Divididos en 12 columnas y contamos con un *universo* de 891 entradas o pasajeros.

In [4]:
print(df.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


También queremos saber cuales son las propiedades de cada entrada, esto está representado en columnas. Ahora que entendemos que tipo de información estamos utilizando podemos comenzar a crear hipótesis y a obtener información *simple* que nos permite crear bloques de datos para su análisis.

Algo importante que recalcar es que todas las bases tienen datos *codificados*, esto quiere decir que un dato en una columna específica tiene un significado, por ejemplo en el caso de nuestros datos la columna *SibSP* puede tener dos tipos de llaves 0 y 1, donde 0 = falso y 1 = verdadero.

Es importante entender esta información antes de comenzar el análisis. En nuestro caso está  codificada de la siguiente forma

| Columna | Definicion | Llave 
| ----------- | ----------- | -----------
| Survival | Sobrevivo? | 0=No, 1=Si
| Pclass | Clase | 	1 = 1st, 2 = 2nd, 3 = 3rd
| Sex | Sexo | female=Mujer, male=Hombre
| Age | Años | 
| Sibsp | #de herman@s / cónyuges a bordo del Titanic
| Parch | #de padres / Niñ@s a bordo del Titanic 
| Ticket | Numero de ticket |
| Fare | Tarifa de pasajero |
| Cbin | Cabina |
| Embarked | Puerto de embarque | C = Cherbourg, Q = Queenstown, S = Southampton

### Limpiando datos
---
Retomando lo comentado en los videos, el paso clave después del análisis previo es la limpieza de datos, dentro de nuestro dataset, no es necesarias las columnas de Nombre, Ticket ni Cabina, ya que esto no nos ayuda a predecir la supervivencia. Por lo tanto los eliminamos de nuestro dataset.

In [5]:
cols = ['Name', 'Ticket', 'Cabin']
df = df.drop(cols, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(2)
memory usage: 62.8+ KB


A continuación, si queremos, podemos eliminar todas las filas de los datos que tienen valores faltantes (NaN). Esto es importante para que no tengamos errores de cálculos futuros

In [6]:
dfna = df.dropna()
dfna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Sex          712 non-null    object 
 4   Age          712 non-null    float64
 5   SibSp        712 non-null    int64  
 6   Parch        712 non-null    int64  
 7   Fare         712 non-null    float64
 8   Embarked     712 non-null    object 
dtypes: float64(2), int64(5), object(2)
memory usage: 55.6+ KB


Después de eliminar las filas con valores NaN **el conjunto de datos se reduce a 712 filas de 891**, lo que significa que **estamos desperdiciando datos**. Por lo tanto, es necesario conservar datos, algo muy común en el análisis de datos es la creación de datos *dummy*, esto quiere decir que utilizamos datos promedio o que podemos encontrar dentro de nuestro mismo data set.
Veamos cual de las columnas tiene mayor cantidad de datos vacíos:


In [7]:
for col in df.columns:
    print(col + ': ' + str( df[col].isna().sum() ))

PassengerId: 0
Survived: 0
Pclass: 0
Sex: 0
Age: 177
SibSp: 0
Parch: 0
Fare: 0
Embarked: 2


Ok, estamos avanzando, la **edad cuenta con 177 campos vacíos** por lo tanto calcularemos la mediana o interpolar todas las edades y llenar esos valores de edad faltantes. Pandas tiene una función interpolar que reemplazará todos los NaN faltantes por valores interpolados.

Puedes leer como mas acerca de esta función en la documentación de pandas

In [8]:
df['Age'] = df['Age'].interpolate()

In [9]:
for col in df.columns:
    print(col + ': ' + str( df[col].isna().sum() ))

PassengerId: 0
Survived: 0
Pclass: 0
Sex: 0
Age: 0
SibSp: 0
Parch: 0
Fare: 0
Embarked: 2


En el caso de Embarked, podemos eliminar los valores vacíos ya que son pocos con respecto al universo que tenemos

In [10]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Sex          889 non-null    object 
 4   Age          889 non-null    float64
 5   SibSp        889 non-null    int64  
 6   Parch        889 non-null    int64  
 7   Fare         889 non-null    float64
 8   Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(2)
memory usage: 69.5+ KB


De esta forma nos quedamos con un universo final de 889 pasajeros. En los siguientes videos comenzaremos a realizar un análisis de datos más profundo pero podemos comenzar a hacer un análisis somero de la información que tenemos por ejemplo podemos obtener el monto total que pagaron los pasajeros del Titanic.

In [11]:
total = df['Fare'].sum()
locale.setlocale( locale.LC_ALL, '' )
print(locale.currency(total, grouping=True ))

$ 28 533.95


Tambien podriamos saber cual es la media de edad 

In [12]:
mean = df['Age'].mean()
print(mean)

29.68044994375703


Esta sección cubre los conceptos más básicos, en las siguientes secciones explicaremos metodos y crearemos nuestro primer algoritmo para calcular la probabilidad de nuestro data set. Continua con nosotros

©BEDU 2020